# Experiments for IF-learner

In [1]:
# set R variables
import os
os.environ['R_HOME'] = 'C:\\Users\\alici\\Anaconda3\\envs\\newenv\\Lib\\R' #path to your R
# installation
os.environ['R_USER'] = 'C:\\Users\\alici\\Anaconda3\\Lib\\site-packages\\rpy2'

from paper_simulations.if_paper.helper_classes import RSmoothingSpline
from paper_simulations.if_paper.if_learner_experiments import eval_range_n, eval_range_bias

In [2]:
range_n = [500, 1000, 3000, 5000, 10000, 30000]

In [3]:
res_n = eval_range_n(RSmoothingSpline(), range_n,  n_jobs=6, 
                     verbose=1)
res_n.to_csv('paper_simulations/if_paper/simulations/CATE_rangen_nobias.csv')

number of train-samples: 500


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   13.7s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   25.4s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   28.0s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


number of train-samples: 1000


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   10.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   24.9s
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:   28.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


number of train-samples: 3000


...

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    8.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   40.1s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.6min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  1.8min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


number of train-samples: 10000


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   16.7s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  2.9min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  3.3min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


number of train-samples: 30000


[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   47.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  3.6min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  8.3min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed:  9.4min finished
